# Source localization pipeline - Stage1
- freesurfer
- .fif-File preparations
 
 


#### Author
Rudi Kreidenhuber <Rudi.Kreidenhuber@gmail.com>
#### License
BSD (3-clause)

## Subject selection

In [ ]:

from src import SubjectDropDowner
import os
import glob
import mne
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

input_dir = "../../MEG"

def get_subject_list(input_dir = input_dir) -> list:
    analist = glob.glob(os.path.join(input_dir, "*"))
    analist = [os.path.basename(f) for f in analist if os.path.isdir(f)]
    fiflist = [os.path.basename(f) for f in glob.glob(os.path.join(input_dir, "*trans_tsss.fif"))]
    fiflist = [f.split("_")[0] for f in fiflist]
    subjectlist = set(fiflist + analist)
    subjectlist = sorted([f for f in subjectlist])
    return subjectlist

subjectlist = get_subject_list()

dl = SubjectDropDowner.SubjectDropDowner(subjectlist)
drop_menu = dl.create_subject_dropdown_widget()
print(f"\n\nThe following patients/ subjects are available:")
drop_menu

# .fif - File preprocessing
- Files are being filtered, downsampled and concatenated
- Files without head transposition and artifact correction via tsss are omitted

## helpers

In [ ]:
def transform_eventfile(event_file):
    """
    Receives a .csv or .txt files as exported i.e. via brainstorm and
    transforms it according to mne-needs.

    Returns a tuple:
    new_eve_file, event_dict
    --> = transformed event-file, dictionary with Eventnames
    """

    eve = pd.read_csv(event_file, header=0)
    le = LabelEncoder()
    labels = eve.iloc[:,0]
    print(f"Labels --> {labels}")
    l_enc = le.fit_transform(labels)
    l_enc = l_enc
    new_eve_file = pd.DataFrame([eve.iloc[:,1], eve.iloc[:,0], (l_enc +1)]).T
    new_eve_file.reset_index(drop=True, inplace = True)
    new_eve_file.iloc[:,0] = (new_eve_file.iloc[:,0]*1000).astype(int)
    new_eve_file.iloc[0,2] = 0  #create one pseudo-event (that is going to be dropped later for some reason)
    new_eve_file.iloc[:,0] = new_eve_file.iloc[:,0].astype(int)
    new_eve_file.iloc[:,1] = new_eve_file.iloc[:,1].astype(str)
    new_eve_file.iloc[:,2] = new_eve_file.iloc[:,2].astype(int)
    new_eve_file.iloc[:,1] = int("0")

    name_of_events = np.unique(eve.iloc[:,0])
    name_of_events = np.sort(name_of_events)
    event_dict=dict()
    event_dict['ignore_me']=0
    for i in range(name_of_events.size):
        key = (name_of_events[i])
        val = i + 1
        event_dict[key] = val
    return new_eve_file, event_dict

def annotate_one_rawfile(raw, rawfile=None, picks="all") -> mne.io.Raw:
    """
    Receives a raw file, finds the corresponding event-file
    Returns an annotated raw file
    """
    eve_name = rawfile.split(".fif")[0] + ".csv"
    if not os.path.isfile(eve_name):
        eve_name = rawfile.split(".fif")[0] + ".txt"
    if os.path.isfile(eve_name): # if fif-file matches event-file --> add events to fif-file
        try:
            print(f"\n\nNow epoching events from {rawfile}\n\n")
            event_file, event_dict = transform_eventfile(eve_name)
            
            annot = mne.annotations_from_events(event_file, sfreq=raw.info["sfreq"], event_desc=event_dict)
            raw.set_annotations(annot)
            print(f"the annot object: {annot}")
            return raw
            
            
            
            
            """
            epochs = mne.Epochs(raw, events=event_file,
                                event_id=event_dict, 
                                tmin=-1.5, tmax=1., 
                                baseline=(-1.5,-0.7), 
                                on_missing = "ignore", 
                                #picks=picks,
                                reject=None,
                                event_repeated="merge",
                                reject_by_annotation=False,  # nothing is dropped, but some epochs are empty
                                )
            #del(raw)
            return epochs
            """
        except Exception as e:
            print(f"failed at annotating events for: {rawfile}\nbecause of {e}")
            return raw
    

In [ ]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    ject = subject
    subject = "sub-" + subject
else:
    subject = subject
    ject = subject.replace("sub-", "")

print(f"Ject: {ject}, Subject: {subject}")

raws = glob.glob(os.path.join(input_dir, "*_trans_tsss.fif"))
raws = [f for f in raws if ject in f]
print(f"The following raw files were found for preprocessing:\n{raws}")

rawfile = raws[0]
raw = mne.io.read_raw_fif(rawfile, preload=True)

annotate_one_rawfile(raw, rawfile)
raw.plot()


In [ ]:
subject = dl.get_subject_dropdown().value

if not subject.startswith("sub-"):
    ject = subject
    subject = "sub-" + subject
else:
    subject = subject
    ject = subject.replace("sub-", "")

print(f"Ject: {ject}, Subject: {subject}")

if not os.path.isdir(os.path.join(input_dir, "processed")):
    os.mkdir(os.path.join(input_dir, "processed"))

output_dir = os.path.join(input_dir, "processed")

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
    print(f"Created output directory: {output_dir}")

target_file = os.path.join(output_dir, str(subject + "_prepped.fif"))

# Make sure we haven't already processed this subject
if os.path.isfile(target_file):
    print(f"\n\nThe file {target_file} already exists, aborting.\n\n")
    raise(FileExistsError)

# Configuration 
# Filter and resample
l_freq: float = 0.1         # lower pass-band edge
h_freq: float = 50.         # higher pass-band edge
fir_design: str = "firwin"  # Filter design method
s_freq: int = 300           # target sampling frequency

# Get all .fif files of the subject
raws = glob.glob(os.path.join(input_dir, "*_trans_tsss.fif"))
raws = [f for f in raws if ject in f]
print(f"The following raw files were found for preprocessing:\n{raws}")

if not raws == []:
    prep_raws = []
    for r in raws:
        raw = mne.io.read_raw(r)
        
        # add events
        
        
        # filter
        raw.load_data()
        raw = raw.filter(l_freq, h_freq, fir_design=fir_design)
        # downsample
        if not raw.info["sfreq"] == s_freq:
            raw = raw.resample(s_freq)
        prep_raws.append(raw)
        del raw

    # concatenate
    try:
        raw = mne.concatenate_raws(prep_raws)
        #annot = mne.Annotations([0, (raw.get_data().shape[1]-1)], [1, 1], ["Start", "bad_ending"], orig_time=raw.info["meas_date"])
        #raw.set_annotations(annot)
        raw.save(target_file, overwrite=False)
    except Exception as e:
        print(f"\n\nSomething went wrong, when trying to concatenate the raw files:\n\nError: {e}")

    del prep_raws

# Freesurfer and hippocampal segmentation

In [ ]:
def get_dicom_file(subject) -> str:
    if not os.path.isdir(os.path.join(input_dir, subject)):
        print(f"No anatomical data found for {subject}, aborting\n\n")
        raise(FileNotFoundError)
    else:
        dicom_path = os.path.join(input_dir, subject, "1*", "100*", "1*", "*")
        dicom = str(glob.glob(dicom_path, recursive=True)[0])
        dicom = os.path.abspath(dicom)
        return dicom

def path_to_wsl(file) -> str:
    file = file.replace("\\", "/")
    return file.replace("c:", "/mnt/c")

def get_dicom_path(subject) -> str:
    dicom = get_dicom_file(subject)
    dicom = path_to_wsl(dicom)
    return dicom

def get_watershed_comand(subject) -> str:
    return f'python -c "import mne; mne.bem.make_watershed_bem(subject=\'{subject}\')"'

def get_recon_all_command(ject) -> str:
    dicom = get_dicom_path(ject)
    subject = "sub-" + ject
    command = f"recon-all -s {subject} -i {dicom} -all && segmentHA_T1.sh {subject} && {get_watershed_comand(subject)}" 
    return command

command = get_recon_all_command(ject)

print(f"\n\nExecute the following command in the bash shell:\n \
      (This will take hours...)\n\n{command}\n\n")

print(f"Meanwhile use brainstorm3 to mark and group spikes on the \
following file:\n\n {os.path.abspath(target_file)}\n\n")

event_file_name = target_file.split(".fif")[0] + ".txt"
print(f"Save the eventfile in the same directory as the raw file as: {os.path.basename(event_file_name)}")
